In [27]:
import numpy as np
import time
import cv2
import os
import imutils

In [28]:
# setting the paths
# custom
# coco_names_path = '../Assets/yolo-coco/custom/classes.names'
# weightsPath = '../Assets/yolo-coco/custom/Training-2/yolov3_custom_last_6000.weights'
# configPath = '../Assets/yolo-coco/custom/yolov3_custom.cfg'

# pretrained
coco_names_path = '../Assets/yolo-coco/coco.names'
weightsPath = '../Assets/yolo-coco/yolov3.weights'
configPath = '../Assets/yolo-coco/yolov3.cfg'

image_path = '../Assets/images/predict-2.jpeg'
video_path = '../Assets/videos/vehicles1.mp4'

In [29]:
# load the COCO class labels our YOLO model was trained on

Labels = []
with open(coco_names_path,'r',encoding='utf8') as f:
    Labels = [line.strip() for line in f.readlines()]

print(Labels)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [30]:
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(Labels), 3),dtype="uint8")

In [31]:
# load our YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that we need from YOLO

net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)
layers_names = net.getLayerNames()
output_layers = [layers_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [32]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(image_path)
writer = None
(W, H) = (None, None)
fps=vs.get(cv2.CAP_PROP_FPS)

In [33]:
# try to determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

[INFO] 1 total frames in video


In [34]:
def get_vehicle_count(boxes, class_names):
    
    total_vehicle_count = 0 # total vechiles present in the image
    dict_vehicle_count = {} # dictionary with count of each distinct vehicles detected
    
    for i in range(len(boxes)):

            class_name = class_names[i]
       
            if(class_name in list_of_vehicles):
                total_vehicle_count += 1
                dict_vehicle_count[class_name] = dict_vehicle_count.get(class_name,0) + 1

    return total_vehicle_count, dict_vehicle_count

In [35]:
# list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle']
list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle', 'ambulance', 'fire engine', 'auto rickshaw']
# looping over the frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(output_layers)
    end = time.time()

    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []
    classname = []
#     print(layerOutputs)
    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            
#             print(detection)
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > 0.4:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
                classname.append(Labels[classID])

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4,0.4)

    
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            
            text = "{}: {:.4f}".format(Labels[classIDs[i]],confidences[i])
            
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    cv2.imshow("frame",frame)
    cv2.waitKey(0)
    # check if the video writer is None
    print("idxs --> ",idxs)
    print("init boxes-->",boxes)
    print("init names-->",classname)
    
    boxes = [boxes[i] for i in idxs]
    classname = [classname[i] for i in idxs]
    
#     print("final boxes -->",boxes)
#     print("final names -->",classname)
    total_vehicles, each_vehicle = get_vehicle_count(boxes, classname)
    print("Total vehicles in image", total_vehicles)
    print("Each vehicles count in image", each_vehicle)
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('../Assets/images/predictedResult.jpeg', fourcc, 30,
            (frame.shape[1], frame.shape[0]), True)

        # some information on processing single frame
        if total > 0:
            elap = (end - start)
            print("[INFO] single frame took {:.4f} seconds".format(elap))
            print("[INFO] estimated total time to finish: {:.4f}".format(
                elap * total))

    # write the output frame to disk
    writer.write(frame)



idxs -->  [ 40  14  66 132  22   8  37  45  27  46  82 108  18  12  96  68  80 126
  85  76 130 137  93 106  56  53 118 103  60  74 136  62 122  89 141 105
 100  58  51   4 144 139  87  88  50 127   2  57 143  52  81  70  61  91]
init boxes--> [[572, 220, 75, 74], [389, 253, 86, 76], [525, 256, 81, 90], [688, 238, 94, 101], [699, 235, 95, 103], [524, 272, 85, 78], [804, 286, 112, 85], [786, 288, 140, 82], [934, 284, 94, 80], [940, 282, 91, 79], [335, 310, 108, 84], [322, 312, 130, 79], [343, 314, 105, 79], [575, 307, 100, 79], [801, 296, 117, 82], [796, 297, 122, 79], [336, 321, 107, 80], [345, 321, 101, 80], [568, 379, 127, 93], [565, 383, 133, 90], [581, 381, 120, 91], [1106, 384, 134, 102], [1100, 387, 142, 93], [1110, 387, 150, 92], [1103, 398, 138, 95], [1101, 400, 144, 92], [241, 446, 128, 96], [766, 430, 114, 107], [748, 435, 146, 97], [1001, 435, 152, 120], [1004, 433, 146, 123], [1005, 429, 142, 129], [1016, 437, 138, 114], [1005, 433, 160, 124], [1005, 432, 159, 124], [226, 4

In [98]:
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...


In [ ]:
car=each_vehicle.get('car',0)
motorbike=each_vehicle.get('motorbike',0)
bicycle=each_vehicle.get('bicycle',0)
bus=each_vehicle.get('bus',0)
truck=each_vehicle.get('truck',0)

In [ ]:
print(car,motorbike,bicycle,bus,truck)

In [ ]:
timer= (truck*3 + bus*3 + car*2 + motorbike*1 + bicycle*2.5)
print(timer)

In [ ]:
# setting the paths
coco_names_path = '../Assets/yolo-coco/custom/classes.names'
weightsPath = '../Assets/yolo-coco/custom/yolov3_custom_last_15600.weights'
configPath = '../Assets/yolo-coco/custom/yolov3_custom.cfg'

image_path = '../Assets/images/predict-3.jpeg'
video_path = '../Assets/videos/vehicles1.mp4'

In [ ]:
import numpy as np
import time
import cv2
image = cv2.imread(image_path)
classes = coco_names_path
with open(classes, 'r') as f:
    LABELS = [line.strip() for line in f.readlines()]
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")
print("[INFO]: Loading image...")
net = cv2.dnn.readNet(weightsPath,configPath)
try:
	(H, W) = image.shape[:2]
	print("[INFO]: Detecting object from image...")
except Exception as inst:
	print('[INFO]: Got Error while loading image:',inst)
	raise SystemExit()

ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()
print("[INFO]: YOLO took {:.6f} seconds".format(end - start))
boxes = []
confidences = []
classIDs = []
for output in layerOutputs:
	for detection in output:
		scores = detection[5:]
		classID = np.argmax(scores)
		confidence = scores[classID]
		if confidence > 0.4:
			box = detection[0:4] * np.array([W, H, W, H])
			(centerX, centerY, width, height) = box.astype("int")
			x = int(centerX - (width / 2))
			y = int(centerY - (height / 2))
			boxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4,0.4)
if len(idxs) > 0:
	for i in idxs.flatten():
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])
		color = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, color, 2)
cv2.imwrite('../Assets/images/predictedResult.jpeg', image)
cv2.imshow("Image", image)
cv2.waitKey(0)

In [ ]:
os.getcwd()

In [ ]:
%run "./keras-yolo3/yolo3/model.py"

In [1]:
import keras
from keras.models import load_model
from tensorflow.keras.layers import Input
import h5py
import io
# Load the pre-trained model
model1 = yolo_body(Input(shape=(416, 416, 3)), 3, 80)

# f = io.open('..', mode="rb")
    
    
model1.load_weights("D:\Projects\Course Projects\MajorProject\RushHour-Traffic-Signal-Models\Assets\yolo-coco\yolov3.h5", by_name=True)

# Load the custom model
model2 = load_model("D:\Projects\Course Projects\MajorProject\RushHour-Traffic-Signal-Models\Assets\yolo-coco\custom\yolov3_custom_last_11900.h5")

# Create an ensemble model
ensemble_model = keras.models.Sequential()
ensemble_model.add(keras.layers.average([model1, model2]))

# Compile the ensemble model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Make predictions using the ensemble model
image = load_image('../Assets/images/bus_cycle_car.jfif')
predictions = ensemble_model.predict(image)


ImportError: cannot import name 'dtensor' from 'tensorflow.compat.v2.experimental' (D:\anaconda3\lib\site-packages\tensorflow\_api\v2\compat\v2\experimental\__init__.py)

In [18]:
import numpy as np
import time
import cv2
import os
import imutils

def get_vehicle_count(boxes, class_names):
    
    list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle']
#     list_of_vehicles = ['car', 'truck', 'bus', 'motorbike', 'bicycle', 'ambulance', 'fire engine', 'auto rickshaw']
    
    total_vehicle_count = 0 # total vechiles present in the image
    dict_vehicle_count = {} # dictionary with count of each distinct vehicles detected
    
    for i in range(len(boxes)):

            class_name = class_names[i]
       
            if(class_name in list_of_vehicles):
                total_vehicle_count += 1
                dict_vehicle_count[class_name] = dict_vehicle_count.get(class_name,0) + 1

    return total_vehicle_count, dict_vehicle_count


def detectVehicles():
    
    coco_names_path = '../Assets/yolo-coco/coco.names'
    weightsPath = '../Assets/yolo-coco/yolov3.weights'
    configPath = '../Assets/yolo-coco/yolov3.cfg'

    image_path = '../Assets/images/predict-1.jpeg'
    video_path = '../Assets/videos/vehicles1.mp4'
    
    # load the COCO class labels our YOLO model was trained on

    Labels = []
    with open(coco_names_path,'r',encoding='utf8') as f:
        Labels = [line.strip() for line in f.readlines()]
        
        
    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(Labels), 3),dtype="uint8")
    
    # load our YOLO object detector trained on COCO dataset (80 classes)
    # and determine only the *output* layer names that we need from YOLO
    net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    # initialize the video stream, pointer to output video file, and
    # frame dimensions
    vs = cv2.VideoCapture(image_path)
    writer = None
    (W, H) = (None, None)
    fps=vs.get(cv2.CAP_PROP_FPS)
    
    # try to determine the total number of frames in the video file
    try:
        prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
            else cv2.CAP_PROP_FRAME_COUNT
        total = int(vs.get(prop))
        print("[INFO] {} total frames in video".format(total))

    # an error occurred while trying to determine the total
    # number of frames in the video file
    except:
        print("[INFO] could not determine # of frames in video")
        print("[INFO] no approx. completion time can be provided")
        total = -1
    
    # looping over the frames from the video file stream
    while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        # construct a blob from the input frame and then perform a forward
        # pass of the YOLO object detector, giving us our bounding boxes
        # and associated probabilities
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(output_layers)
        end = time.time()

        # initialize our lists of detected bounding boxes, confidences,
        # and class IDs, respectively
        boxes = []
        confidences = []
        classIDs = []
        classname = []
    # print(layerOutputs)
        # loop over each of the layer outputs
        for output in layerOutputs:
            # loop over each of the detections
            for detection in output:
                # extract the class ID and confidence (i.e., probability)
                # of the current object detection

    # print(detection)
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]

                # filter out weak predictions by ensuring the detected
                # probability is greater than the minimum probability
                if confidence > 0.5:
                    # scale the bounding box coordinates back relative to
                    # the size of the image, keeping in mind that YOLO
                    # actually returns the center (x, y)-coordinates of
                    # the bounding box followed by the boxes' width and
                    # height
                    box = detection[0:4] * np.array([W, H, W, H])
                    (centerX, centerY, width, height) = box.astype("int")

                    # use the center (x, y)-coordinates to derive the top
                    # and and left corner of the bounding box
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))

                    # update our list of bounding box coordinates,
                    # confidences, and class IDs
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
                    classname.append(Labels[classID])

        # apply non-maxima suppression to suppress weak, overlapping
        # bounding boxes
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.5)


        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                (x, y) = (boxes[i][0], boxes[i][1])
                (w, h) = (boxes[i][2], boxes[i][3])

                # draw a bounding box rectangle and label on the frame
                color = [int(c) for c in COLORS[classIDs[i]]]

                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

                text = "{}: {:.4f}".format(Labels[classIDs[i]],confidences[i])

                cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.imshow("frame",frame)
        cv2.waitKey(0)
        # check if the video writer is None
#         print("idxs --> ",idxs)
#         print("init boxes-->",boxes)
#         print("init names-->",classname)

        boxes = [boxes[i] for i in idxs]
        classname = [classname[i] for i in idxs]

    #     print("final boxes -->",boxes)
    #     print("final names -->",classname)
        total_vehicles, each_vehicle = get_vehicle_count(boxes, classname)
        print("Total vehicles in image", total_vehicles)
        print("Each vehicles count in image", each_vehicle)
        if writer is None:
            # initialize our video writer
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter('../Assets/images/predictedResult.jpeg', fourcc, 30,
                (frame.shape[1], frame.shape[0]), True)

            # some information on processing single frame
            if total > 0:
                elap = (end - start)
                print("[INFO] single frame took {:.4f} seconds".format(elap))
                print("[INFO] estimated total time to finish: {:.4f}".format(
                    elap * total))

        # write the output frame to disk
        writer.write(frame)

    # release the file pointers
    print("[INFO] cleaning up...")
    writer.release()
    vs.release()
    
    car=each_vehicle.get('car',0)
    motorbike=each_vehicle.get('motorbike',0)
    bicycle=each_vehicle.get('bicycle',0)
    bus=each_vehicle.get('bus',0)
    truck=each_vehicle.get('truck',0)
    
    
    return each_vehicle
    
    

In [19]:
detectVehicles()

[INFO] 1 total frames in video
Total vehicles in image 64
Each vehicles count in image {'car': 58, 'truck': 5, 'bus': 1}
[INFO] single frame took 0.5106 seconds
[INFO] estimated total time to finish: 0.5106
[INFO] cleaning up...


{'car': 58, 'truck': 5, 'bus': 1}